In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('int64'), y_test.astype('int64')

x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test /255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

In [5]:
w = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))

In [6]:
@tf.function
def softmax_regression(x):
  logits = tf.matmul(x, w) + b
  return tf.nn.softmax(logits), logits

In [7]:
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [9]:
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), y)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

optimizer = tf.optimizers.SGD(0.5)

In [10]:
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = softmax_regression(x)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, [w,b])
  optimizer.apply_gradients(zip(gradients,[w,b]))

In [12]:
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(batch_xs, batch_ys)

In [13]:
print("정확도(Accuracy): %f" % compute_accuracy(softmax_regression(x_test)[0], y_test))

정확도(Accuracy): 0.918000
